# 1	Introduction
The MULTIPLY framework is a new toolkit to extract information from remote sensing data. In contrast to other remote sensing software packages, MULTIPLY is designed to consistently use heterogeneous satellite data (coarse vs high, optical vs microwave). Instead of multiple independent process-chains for each individual satellite, MULTIPLY is a single all-in-coupling framework in order to:
*	Create consistencies between different land surface products (reducing errors in downstream-processing)
*	Utilize the benefits of multiple satellites (including Sentinel-1, Sentinel-2, MODIS, Landsat), such as 
    * Higher temporal coverage
    * Spectral Sensitivity to more land surface variables
*	Quantify errors of output products, depending on input uncertainties.
*	Enable operational service for different end-users, including: 
    * Remote sensing consultants
    *	Remote sensing experts
In order to achieve these goals MULTIPLY uses a data-assimilation framework developed for the ESA (Gómez-Dans et al. 2016; Lewis et al. 2012). In this framework the state-vector (describing the land surface parameters of interest) are coupled to the observations using radiative transfer models. In addition priori-information (obtained from field-measurements databases) are used to constrain the final results. 


# 2	MULTIPLY Concepts
## 2.1	Radiative transfer models
In it’s most basic description, a radiative transfer model is a model  capable of (using a specific state-vector) as an input simulateing the different interactions of radiation (light) from source (sun/clouds/..) to sensor (satellite ), using a specific state-vector as an input.  Within the model the most important absorption/transmission and reflection processes by different objects (soil/leaf/atmosphere) is characterized. In this regard there exist many radiative transfer models exist, depending on the level and detail of which important processes are taken into account. In practice, highly complex radiative radiative transfer models usually have a high number of input parameters (to describe in detail the 3D structure of the land surface and the full land surface heterogeneity), while less complex models employ specific hypotheses (such as land surface homogeneity) to require less input parameters. 

<img src="pics/BasicDescriptionRTM2.png">
Figure 1: Basic description of Radiative Transfer model

While this produces a large variety in radiative transfer models, all of these models have at least one common aspect: they can only be run in ‘forward mode’. This is because completely different scenarios can produce similar remote sensing observations. For example, when a land surface has a very low fractional vegetation cover (~0), different chlorophyll levels will produce similar remote sensing observations.

## 2.2	Data assimilation
The most common way to circumvent the shortcoming of radiative transfer models is to ‘guess’ the initial state-vector, run the radiative transfer model using this initial assumption, and compare the output observation against the remote sensing measurement, producing an error. By guessing multiple times,  (for instance by constructing a Look-up-Table (LUT), multiple error-values can be calculated. The lowest error-value than represents our best understanding of the land surface (as long as our initial assumption was close enough to the real land surface description). 

<img src="pics/CreatingLUTs.png">
Figure 2: Creating LUT tables

Such an LUT approach has the large disadvantage that it only finds values that were used in the initial guessing; for example if for the Leaf Area Index (LAI) only values [0, 1, 2,.. ,6] were used, this retrieval method will not find the (actual) value of 1.22 m2/m2. 
The shortcoming of the LUT approach can be greatly to a large part be circumvented if the initial scenarios can be updated on basis of the errors found,  (together with taking into account the sensitivity of the model to the state vector), as illustrated by in Figure 3. Such an iterative approach is in general ‘optimization’ of the radiative transfer model. 
 
<img src="pics/BasicOptimization.png">
Figure 3: Basic description of Optimization methodology

In this approach, it is possible for the state-vector to ‘slowly’ drift away from our prior-value. In addition, the approach is limited to produce only results for when satellite measurements are being performed. In order to solve these shortcomings additional (prior) information can be used within a data-assimilation framework. This is illustrated by Figure 4.

<img src="pics/DataAssimilation.png">
Figure 4: Data Assimilation of Radiative Transfer Models

## 2.3	Prior Information
In such an data-assimilation approach, an additional error (Error value 2) different from the observational error (Error 1) is defined, describing the deviation of the state-vector from our initial assumption. Here the observational error is defined in units of the observation (e.q. reflectances [-] or radiances [W sr^(-1) m^(-2) nm^(-1)]), while the prior-error is defined in units of the state vector (LAI  [m^2  m^(-2)], Leaf Angle Distribition [deg], …). In order to reconcile these two into one cost function the errors are normalized. For the observational error this is performed by considering the observational errors, while for the prior information this is accomplished by considering the prior uncertainty (for example the standard deviation of long-term measurements of the respective land surface parameter). This leads then to one single cost-function defined as:

J_tot=(Error_1)/σ_1 +  (Error_2)/σ_2 =J_obs+J_prior

This also immediately provides us with a method on synergistically combining different types of observations, as the observational error can simply be expanded to include multiple observations

J_obs=J_obs^1+ J_obs^2+ J_obs^3+⋯

In short, each of the error-terms corresponds to different pieces of information added to the system. In this case, even more additional information can also be introduced to the system, be considering the temporal evolution and spatial variation of specific land surface parameters. 


# 3	The prototype version overview 
We hereby provide you with a working prototype  version. This prototype  version is a stripped down version of the actual framework focusing primarily on the optical retrieval using both coarse (MODIS) and high resolution (Sentinel-2) observations. An overview of the full framework is provided in Figure 6. Here the specific modules that were not mature to be incorporated in this release, are the SAR preprocessing, SAR integration, Post processing (for biodiversity and fire disturbance monitoring), as well as the visualization component (highlighted by red circles )

<img src="pics/MultiplyModules.png">
Figure 5: Overview of MULTIPLY framework. Modules highlighted by a red circle are not included in the prototype version

In the version you will be able to download al the required earth observation data (using the data-access component), and afterwards retrieve land surface parameters (using the Inference Engine Module) at  20m resolution using a combination of MODIS coarse resolution data (from the Coarse res Preprocessing module) together with high resolution Sentinel-2 observations (from the high res preprocessing module), together with prior information on the land surface (by the Prior Engine module). In the future it is foreseen that multiple Radiative transfer models can be used, specific to the user requirements, however at the moment the only one available (in the forward operator module) is the emulated version of the PROSAIL radiative transfer mode . This limits the retrieval of land surface parameters to a) leaf-traits (Chlorophyll, Carotonoids, Dry matter, Water Content),  b) Vegetation structural parameters (hotspot parameter, LAI, leaf structure), and c some auxiliary soil parameters (Bsoil/Psoil).

<img src="pics/MultiplyOutput.png">
Figure 6: Overview of state-vector parameters retrievable on basis of the underlying forward operator (radiative transfer model

# 4 Usage
All the code for the individual modules are is located at https://github.com/multiply-org/. This can be used to setup the MULTIPLY framework on your own computing infrastructure. At present however no deployment setup (in the form of windows-setup-executables, or anaconda package’s) exist. While this is planned further intolater in the project, the focus at this stage is on testing the individual components themselves. Please let us know if you would prefer to install the software yourself on a dedicated computational framework, so that we can investigate how to facilitate this for you. 
In order to facilitate the testing of the framework itself, we have setup this Virtual Machine on Google Compute Engine, for testing purposes. 


###### Please note that for the default setup (currently shown in this python notebook), including downloading the data, atmospherically correcting it, creating user-defined priors and performing the the complete inference (over a single day) for the Jarvjelja Forest site in Estonia, takes ... minutes to complete. 


## 4.0 Load internal packages and auxiliary methods

In [ ]:
import os
from multiply_data_access import DataAccessComponent
from vm_support import create_sym_links
import datetime
import glob
from typing import List, Optional
from vm_support import create_config_file, set_permissions
from multiply_prior_engine import PriorEngine

tic = datetime.datetime.now()

## 4.1 Defining the interfaces to work with the MULTIPLY framework
First we need to setup interfaces to work and test with the MULTIPLY framework. In actual operation, these auxiliary interfaces are bundled together within a single python class. However as we would like you to test the different modules individually, we provide direct access to these. For more information on how these interfaces are defined, please check the following link: [MULTIPLY Tools](Tools.py). 


In [ ]:
from vm_support.tools import get_static_data, get_dynamic_data

In [ ]:
from vm_support.tools import preprocess

In [ ]:
from vm_support.tools import get_priors, get_priors_from_config_file

In [ ]:
from vm_support.tools import infer, infer3, infer_new

In [ ]:
from vm_support.tools import create_dir, put_data, InvTransformation

In [ ]:
from vm_support.tools import Plot_SRDS,Plot_PRIORS, Plot_TRAITS,Plot_TRAIT_evolution, Plot_Transformation

# 5 Running MULTIPLY
Below the actual code is provided for running the MULTIPLY framework.
We start with setting earth data authentication. This is required to download the MODIS brdf descriptors which are required for the atmospheric correction of the Sentinel-2 data. You can get credentials when you register at https://urs.earthdata.nasa.gov/profile . Registration and use is free of cost. If you do not register, you can only use the MODIS data which has been downloaded in previous runs of the notebook by other users.
Also you will need to set up the data stores so that the data access component is working correctly and finds the pre-configured data stores. Both steps only need to be performed once.

In [ ]:
# from vm_support import set_earth_data_authentication, set_up_data_stores
# set_up_data_stores()
# username = ''
# password = ''
# set_earth_data_authentication(username, password) # to download modis data, needs only be done once

## 5.1 Parameters
Here you can actually set the parameters for the run. The parameters are as follows:
* **roi**: A region of interest, given as a Polygon in WKT format. You can use this tool ( https://arthur-e.github.io/Wicket/sandbox-gmaps3.html ) to easily get definitions of the regions you are interested in in WGS84 coordinates.
* **roi_grid**: The EPSG-code of the spatial reference system in which the roi is given. If it is set to 'none', it is assumed that the roi is given in WGS84 coordinates.
* **destination_grid**: The EPSG-code of the spatial reference system in which the output shall be given. If it is set to 'none', the platform will attempt to derive it from the roi_grid.
* **spatial_resolution**: The resolution the output data is supposed to have, must be a non-negative integer number. The resolution is given in meters and is the same for both dimensions.
* **start_time**: The start date of the period you are interested in, must be given in the format 'Year-Month-Day' as below.
* **end_time**: The end date of the period you are interested in, must be given in the format 'Year-Month-Day' as below.
* **time_step**: The temporal resolution the output is supposed to have. Data will be aggregated over the period denoted by this parameter. Must be a non-negative integer value. The unit is days.
* **variables**: The list of the biophysical variables that shall be derived. Please do not change this list, as the underlying forward model requires all of them. The parameters are as follows:
  * **n**: Structural parameter
  * **cab**: Leaf Chlorophyll Content, given in ug/cm²
  * **car**: Leaf Carotonoid Content, given in ug/cm²
  * **cb**: Leaf senescent material
  * **cw**: Leaf Water Content, given in cm
  * **cdm**: Leaf Dry Mass, given in g/cm²
  * **lai**: Effective Leaf Area Index, given in m²/m²
  * **ala**: Average Leaf Angle, given in degrees
  * **bsoil**: Soil Brightness Parameter
  * **psoil**: Soil Wetness Parameter
* **file_mask**: A file that can be used to explicitly state the region you are interested in. You can also use it to mask out single pixels within this region. If this is not 'none', the aforementioned parameters roi_grid, spatial_resolution, and destination_grid are not used.

**HINT**: The platform will perform faster the smaller your roi and the larger the spatial resolution is.

### 5.1.1 Define region of Interest

In [ ]:
# tartu
roi_grid = 'EPSG:4326'
roi_centroid = [27.3, 58.3]
roi =  'POLYGON((27.1615456795 58.2252523466, ' + \
                '27.1720041127 58.3418423196, ' + \
                '27.3885799673 58.3362682904, ' + \
                '27.3774098023 58.2196966529, ' + \
                '27.1615456795 58.2252523466))'

# montesinho
# roi_grid = 'EPSG:4326'
# roi_centroid = [-6.749181264648428, 41.84622613151897]
# roi =  'POLYGON((-6.93 42.00, ' + \
#                 '-6.48 42.00, ' + \
#                 '-6.48 41.68, ' + \
#                 '-6.93 41.68, ' + \
#                 '-6.93 42.00))'

# roi_centroid = [-3.50, 37.60]
# roi  = 'POLYGON((2.20 37.98, ' + \
#                 '2.40 37.98, ' + \
#                 '2.40 37.35, ' + \
#                 '2.20 37.35, ' + \
#                 '2.20 37.98))'

# roi_centroid = [353.20, 41.84]
# roi =  'POLYGON((353.07 42.00, ' + \
#                 '353.52 42.00, ' + \
#                 '353.52 41.68, ' + \
#                 '353.07 41.68, ' + \
#                 '353.07 42.00))'


destination_grid = 'EPSG:3301'                
spatial_resolution = 20 # in m

In [ ]:
from IPython.display import IFrame
from IPython.core.display import display

google_maps_url = "http://maps.google.com/maps?q=48.184543+11.213&ie=UTF8&t=h&z=18&output=embed&z=17"
google_maps_url = 'http://maps.google.com/maps?q=%7.5f' % roi_centroid[1] + '+%7.5f' % roi_centroid[0] +'&ie=UTF8&t=h&z=18&output=embed&z=10'
IFrame(google_maps_url,800,600)




### 5.1.2 Define temporal frequency and period

In [ ]:
# start_time_as_string = '2018-05-8'
# stop_time_as_string = '2018-05-9'

# tartu
start_time_as_string = '2018-07-24'
stop_time_as_string = '2018-07-25'

# montesinho
# start_time_as_string = '2019-06-29'
# stop_time_as_string = '2019-07-30'



# start_time_as_string = '2018-05-13'
# stop_time_as_string = '2018-05-14'

time_step = 10 # in days

start_time_as_datetime = datetime.datetime.strptime(start_time_as_string, '%Y-%m-%d')
stop_time_as_datetime = datetime.datetime.strptime(stop_time_as_string, '%Y-%m-%d')

time_step_as_time_delta = datetime.timedelta(days=time_step)
variables = {'n', 'cab', 'car', 'cb', 'cw', 'cdm', 'lai', 'ala', 'bsoil', 'psoil'}

# file_mask = "mask.tif"
file_mask = None

### 5.1.3 Setting up the working directory
For this notebook, you will operate in your own working directory. All data you use will be copied here, all output will be written here.

In [ ]:
from vm_support import get_working_dir
name = 'm1_again51EE'
# use previous (non-empty) working directory
working_dir = '/Data/test_user_16/' + name

#clear working directory
working_dir = get_working_dir(name)



In [ ]:
print('Working directory is {}'.format(working_dir))

priors_directory = '{}/priors'.format(working_dir)
hres_state_dir = '{}/hresstate'.format(working_dir)
modis_directory = '{}/modis'.format(working_dir)
state_directory = '{}/state'.format(working_dir)
cams_directory = '{}/cams'.format(working_dir)
s2_l1c_directory = '{}/s2'.format(working_dir)
sdrs_directory = '{}/sdrs'.format(working_dir)
biophys_output = '{}/biophys'.format(working_dir)
emulators_directory = '{}/emulators'.format(working_dir)
dem_directory = '{}/dem'.format(working_dir)

## 5.2 Acquire Static Data
We differentiate between two types of data here: Dynamic and static, meaning: Data which are valid for a certain period of time and data which is valid permanently. The latter is the elevation data of the Digital Elevation Model and the Emulators required for the Atmospheric Correction. We put them in their designated folders before we start our loop through time.

In [ ]:
data_access_component = DataAccessComponent()
get_static_data(data_access_component=data_access_component, roi=roi,
                start_time=start_time_as_string, stop_time=stop_time_as_string, 
          emulation_directory=emulators_directory, dem_directory=dem_directory, roi_grid=roi_grid)

# 6. Infer variables
Now we can actually infer the bio-physical variables. For this, we will step though the time grid that we have set up using the start_time, end_time and time_step parameters above.
For this stepping, we create a *cursor* that points to the beginning of the time period that we are currently deriving data for.
We also define two variables *previous_inference_state* and *updated_inference_state* which will save the state of the inference engine and will ensure that the inference engine can consider the results from its last run.
The variable *preprocess_only_region_of_interest* allows to preprocess only the portion of the S2 image that we will consider during the inference. If we process the whole image though, we can save it and use it in later runs.

Thes stepping of time works as follows: Dedicated directories are set up for the MODIS, CAMS, and S2 data. These data are retrieved and put into these directories. After that, pre-processing will take place either on the whole S2 image or on the region of interest. If it has been performed on the whole region, the result will be permanently saved. Next, priors will be derived for every variable and every day within the current period. Having gathered all these, the inference can finally begin. The state of the inference engine is saved and considered during the next iteration.

## 6.1 Single Run Test
First for explanation, we will perform the inference over a single timestep. During this, we will show all the results to indicate the flow of the processings. 

### 6.1.1 setting up

In [ ]:
cursor = start_time_as_datetime
previous_inference_state = None #'none'
updated_inference_state = 'none'
one_day_step = datetime.timedelta(days=1)
preprocess_only_region_of_interest = True

date_as_string = datetime.datetime.strftime(cursor, '%Y-%m-%d')
print('Doing time step starting on {}'.format(date_as_string))

In [ ]:
cursor += time_step_as_time_delta
cursor -= one_day_step
if cursor > stop_time_as_datetime:
    cursor = stop_time_as_datetime
next_date_as_string = datetime.datetime.strftime(cursor, '%Y-%m-%d')
cursor += one_day_step
cursor_as_string = datetime.datetime.strftime(cursor, '%Y-%m-%d')

modis_directory_for_date = '{}/{}'.format(modis_directory, date_as_string)
cams_directory_for_date = '{}/{}'.format(cams_directory, date_as_string)
s2_l1c_directory_for_date = '{}/{}'.format(s2_l1c_directory, date_as_string)
sdrs_directory_for_date = '{}/{}'.format(sdrs_directory, date_as_string)
priors_directory_for_date = '{}/{}/'.format(priors_directory, date_as_string)

In [ ]:
cursor = start_time_as_datetime
date_as_string = datetime.datetime.strftime(cursor, '%Y-%m-%d')
priors_directory_for_date = '{}/{}/'.format(priors_directory, date_as_string)


### 6.1.2 Get Data

In [ ]:
get_dynamic_data(data_access_component, roi, roi_grid, date_as_string, next_date_as_string,
                 modis_directory_for_date, cams_directory_for_date, s2_l1c_directory_for_date)

### 6.1.3  Define Priors
You can choose to have the system create it's own 'default' configuration file (using the parameter values, specified earlier). Alternatively, one can make user-specific requests. For example, One can alter the Priors that are going to be used in the inference. If you would like to use these more complex metod, in the actual inference you should provide a link to this user configuration file (or uncomment this line, if one prefers the default configuration). 

Please be aware that new files have to be created in case of user-defined priors, which can take some time (depending on number of 'user-defined' priors, the size of the area and the duration of the time-period). 

In [ ]:
# Setting up the default configuration (of flat priors)
config_file = None
user_priors = {}
user_priors['lai'] = {'mu': 3.0, 'unc': 0.2}
# user_priors['cab'] = {'mu': 70.0, 'unc': 70.*0.0001}
# user_priors['car'] = {'mu': 10.0, 'unc': 10.0*0.01}
# user_priors['cw'] = {'mu': 0.005, 'unc': 0.005*0.01}
# user_priors['cdm'] = {'mu': 0.0035, 'unc': 0.0035*0.01}
# user_priors['cb'] = {'mu': 0.01, 'unc': 0.01*0.01}
# user_priors['n'] = {'mu': 1.6, 'unc': 1.6*0.01}
# user_priors['ala'] = {'mu': 70., 'unc': 70*0.01}

# user_priors['psoil'] = {'mu': 70., 'unc': 70*0.01}

# config_file = None
# user_priors = {}
# user_priors['lai'] = {'mu': 3.00, 'unc': 0.02}
# user_priors['cab'] = {'mu': 70.0, 'unc': 70.*0.0001}
# user_priors['car'] = {'mu': 10.0, 'unc': 10.0*0.01}
# user_priors['cw'] = {'mu': 0.005, 'unc': 0.005*0.01}
# user_priors['cdm'] = {'mu': 0.0035, 'unc': 0.0035*0.01}
# user_priors['cb'] = {'mu': 0.01, 'unc': 0.01*0.01}
# user_priors['n'] = {'mu': -1.6, 'unc': 1.6*0.01}
# user_priors['ala'] = {'mu': 70., 'unc': 70*0.01}

# user_priors = {}
# user_priors['lai'] = {'mu': -0.2, 'unc': 0.02}
# user_priors['cab'] = {'mu': -0.6, 'unc': 0.6*0.01}
# user_priors['car'] = {'mu': -0.9, 'unc': 0.9*0.01}
# user_priors['cw']  = {'mu': -0.6, 'unc': 0.6*0.01}
# user_priors['cdm'] = {'mu': -0.7, 'unc': 0.7*0.01}
# user_priors['cb']  = {'mu': -0.2, 'unc': 0.2*0.01}
# user_priors['n']   = {'mu': -1.6, 'unc': 1.6*0.01}
# user_priors['ala'] = {'mu': -0.9, 'unc': 0.9*0.01}


# link to user-configuration file. Comment out to create  default configuration file
# config_file = '/home/test_user_16/config_user.yaml'
# Please be aware that the working directory in the configuration file should be the same as the working directory 
# specified above


In [ ]:
if config_file is not None:
#     print('choice 1')
    get_priors_from_config_file(date_as_string, next_date_as_string, priors_directory_for_date, variables, config_file)
else:
#     print('choice 2')
    get_priors(working_dir, roi, date_as_string, next_date_as_string, time_step, priors_directory_for_date, variables, user_priors)
    config_file = working_dir + '/config.yaml'


In [ ]:
# visualize Priors
Plot_PRIORS(roi_centroid,priors_directory_for_date,variables,2)

In [ ]:
# u=0
# # TEST!!!
# user_priors2 = {}
# for t in range(Nt):
#     varname = list(variables)[t]
#     user_priors2[varname] = {'mu':0,'unc':0}#         a = 
#     user_priors2[varname]['mu'] = -float(classes_u[t,u])      # to indicate priors are already transformed
#     user_priors2[varname]['unc'] =  float(classes_u_unc[t,u]) # float(0.000001) #classes_u_unc[t,u]

# # modify prior-engine to take 'transformed values directly!!'
# get_priors(working_dir, roi, date_as_string, next_date_as_string, time_step, priors_directory_for_date, variables, user_priors2)

# # visualize Priors
# Plot_PRIORS(roi_centroid,priors_directory_for_date,variables,2)

### 6.1.4 Test Retrieval without Observations

In [ ]:
infer_new(config_file, date_as_string, cursor_as_string, None, priors_directory_for_date, sdrs_directory_for_date, 
          updated_inference_state, biophys_output,variables, None, spatial_resolution, roi_grid,destination_grid)

In [ ]:
import gdal 
import numpy as np
import matplotlib.pyplot as plt
import glob

print(biophys_output)
variables_subset = variables
date = '205'
Plot_TRAITS(biophys_output,date,variables_subset)

#### 6.1.4.1 Segment Priors into different classes

In [ ]:
from vm_support.tools import Classify_PRIORS3
Iclass, classes_u, classes_u_unc = Classify_PRIORS3(biophys_output,variables,date,0.1)

Iclass[Iclass<0] = np.nan
Nt,Nu = np.shape(classes_u)


In [ ]:
# from Tools import Read_TraitPriors2
# Data_unc = Read_TraitPriors2(biophys_output,variables, date,'_unc')
# Data_unc = np.array(Data_unc)
# print(list(variables)[0])
# plt.figure(figsize=[20,10])
# plt.subplot(1,2,1)
# plt.imshow(Data_unc[0,:,:])
# plt.subplot(1,2,2)
# plt.imshow(Iclass==1)

In [ ]:
plt.figure(figsize=[5,5])
plt.imshow(Iclass)
plt.colorbar()
plt.title('total number of unique Priors = %d' % np.shape(classes_u)[1])

plt.figure(figsize=[20,5])
plt.subplot(1,3,1)
for t,varname in enumerate(variables):
    plt.plot(InvTransformation(varname,classes_u[t,:]))
plt.title('Prior untransformed values of classes')

plt.subplot(1,3,2)
for t in range(Nt):
    plt.plot(classes_u[t,:])
plt.title('Prior transformed values of classes')


plt.subplot(1,3,3)
for t in range(Nt):
    plt.plot(classes_u_unc[t,:])
plt.legend(variables)
plt.title('Prior uncertainties of classes')

In [ ]:
# correct for large variation in uncertainties
for t in range(Nt):
    # print(np.abs(classes_u_unc[t,]) > (np.mean()+ np.std(classes_u_unc[t,])))
    v = classes_u_unc[t,]
    p = np.percentile(v,[34, 50, 68])
    v[(v<p[0]) + (v>p[2])] = p[1]
    classes_u_unc[t,] = v
    

### 6.1.5 Preprocessing Observations

In [ ]:
from vm_support.tools import Check_SDRS_Files

In [ ]:
# get_dynamic_data(data_access_component, roi, roi_grid, date_as_string, next_date_as_string,
#                  modis_directory_for_date, cams_directory_for_date, s2_l1c_directory_for_date)
preprocess(s2_l1c_directory_for_date, modis_directory_for_date, emulators_directory, cams_directory_for_date, 
           dem_directory, sdrs_directory_for_date, roi)
Check_SDRS_Files(sdrs_directory_for_date)

In [ ]:
# visualize the atmospheric correction
Productnr = 0
Plot_SRDS(sdrs_directory, Productnr)

In [ ]:
#Productnr = 1
#Plot_SRDS(sdrs_directory, Productnr)


In [ ]:
Check_SDRS_Files(sdrs_directory_for_date)

### 6.1.6  Retrieval with Observations
Due to an instability in the inferenc engine (that is currently being investigated). The inference only works with flat priors. Therefore we first run (above) the inference with purely priors. This will results in biophysical parameters for the selected area to be equivelent to the prior values. Afterwards we segment the retrieval to be run for individual prior-classes. Basically then we 
1) require priors to be segmented into different classes 
2) for each class,  use the 'user' option to define flat priors with the appropriate mean/unc values
3) run the inference
4) merge everything together. 


#### 6.1.6.1 Run the inference for the different classes

In [ ]:
# Nt, Nu = np.shape(classes_u)
# classes_u = float(classes_u)
# classes_u_unc = float(classes_u_unc)

# indicate that the priors are already 'transformed' 
# classes_u_t =  - classes_u

In [ ]:
Nx, Ny = np.shape(Iclass)
Nx, Ny, = np.shape(Iclass)
Traits = np.zeros((Nx,Ny,Nt,Nu))
Traits_unc = np.zeros((Nx,Ny,Nt,Nu))


In [ ]:
from vm_support.tools import Read_TraitPriors2

for u in range(Nu):
    # define flat_priors
    user_priors2 = {}
    for t in range(Nt):
        varname = list(variables)[t]
        user_priors2[varname] = {'mu':0,'unc':0}#         a = 
        user_priors2[varname]['mu'] = -float(classes_u[t,u])      # to indicate priors are already transformed
        user_priors2[varname]['unc'] =  float(classes_u_unc[t,u]) # float(0.000001) #classes_u_unc[t,u]
        
    # modify prior-engine to take 'transformed values directly!!'
    get_priors(working_dir, roi, date_as_string, next_date_as_string, time_step, priors_directory_for_date, variables, user_priors2)

    # run inference
    infer_new(config_file, date_as_string, cursor_as_string, None, priors_directory_for_date, sdrs_directory_for_date, 
          updated_inference_state, biophys_output,variables, None, spatial_resolution, roi_grid,destination_grid)

    # read retrieved values + uncertainty
    Data = Read_TraitPriors2(biophys_output,variables, date, str='')
    Data_unc = Read_TraitPriors2(biophys_output,variables, date, str='_unc')
    Data = np.array(Data)
    Data_unc = np.array(Data_unc)
    Traits[:,:,:,u] = np.transpose(Data,(1,2,0))
    Traits_unc[:,:,:,u] = np.transpose(Data_unc,(1,2,0))
Traits2 = Traits*1.
Traits_unc2 = Traits_unc*1.

In [ ]:
# visualize construction of final results from prior-classes
t=0 
varname = list(variables)[t]

plt.figure(figsize=[20,15])
for u in range(Nu):
    V = Traits[:,:,t,u]*(Iclass==u)
    V[V==0]=np.nan
    V = InvTransformation(varname,V)
    
    plt.subplot(2,4,u+1)
    plt.imshow(V)
    plt.colorbar()
    plt.title('Class = {}'.format(u))

In [ ]:
# Merge traits into 1 map
Trait = np.zeros((Nx,Ny,Nt))
Trait_unc = np.zeros((Nx,Ny,Nt))
for t,varname in enumerate(variables):
    for u in range(Nu):
        Trait[:,:,t]     = Trait[:,:,t]     + (Iclass==u)*Traits[:,:,t,u]
        Trait_unc[:,:,t] = Trait_unc[:,:,t] +  (Iclass==u)*Traits_unc[:,:,t,u]
    
# visualize
t=0
varname = list(variables)[0]
plt.figure(figsize=[20,5])
plt.subplot(1,2,1)
plt.imshow(Trait[:,:,t])
plt.colorbar()
plt.title('Retrieved {}'.format(varname))

plt.subplot(1,2,2)
plt.imshow(Trait_unc[:,:,t])
plt.colorbar()
plt.title('Uncertainties {}'.format(varname))

#### 6.1.6.2 Store merged traits values

In [ ]:
for t,varname in enumerate(variables):    
    data_file = glob.glob(biophys_output+ '/' + varname + '*'+ date +  '' + '.tif')[0]
    data_set = gdal.Open(data_file, gdal.GA_Update)
    data_set.GetRasterBand(1).WriteArray(Trait[:,:,t])
    data_set = None

    data_file_unc = glob.glob(biophys_output+ '/' + varname + '*'+ date +  '_unc' + '.tif')[0]
    data_set = gdal.Open(data_file, gdal.GA_Update)
    data_set.GetRasterBand(1).WriteArray(Trait_unc[:,:,t])
    data_set = None

#### 6.1.6.3  Visualize retrieval

In [ ]:
import gdal 
import numpy as np
import matplotlib.pyplot as plt
import glob

print(biophys_output)
variables_subset = variables
date = '205'

Plot_TRAITS(biophys_output,date,variables_subset)    

In [ ]:
toc = datetime.datetime.now()
T = (toc - tic).seconds
days = np.floor(T/60./60./24)
res = T - days*60.*60.*24.
hours = np.floor(res/60./60.)
res = T - hours
minutes = np.ceil(res/60.)

print('The run took %02d:%02d:%02d minutes ' % (days,hours,minutes))


## 6.2 Read Data 
All the variables are stored within the user directory. However for visualisation purposes we here only provide you to investigate a 'single variable of interest. Please find all the output files located at the following directory:

In [ ]:
print(biophys_output)

In [ ]:
import gdal
import numpy as np


variable_of_interest = 'lai'

data_files = set(glob.glob(biophys_output + '/*' + variable_of_interest + '*')) - \
            set(glob.glob(biophys_output + '/*unc*'))
num_of_data_sets = len(data_files)

Data = []
print('reading %s files into memory' % variable_of_interest)
for data_file in data_files:
    data_set = gdal.Open(data_file)
    data = data_set.ReadAsArray(0)
    data[np.where(data<1e-8)]=np.NaN
    data[np.where(data>0.99)]=np.NaN
    
    Data.append(data)    
print('Finished reading all %s files' % variable_of_interest)    

data_files

## 6.3 Process Data
Within MULTIPLY we use transformation on some of the variables in order to (approximately) linearize the  radiative transfer model. In this sense, LAI within the state-vector is corrected for the exponential extinction of radiation that occurs witin the canopy. 

LAI = exp(LAI/-2.0)

This linearisation speeds-up the inference and also allows us to propagate the uncertainties better. However it comes at the expense that the output results of the inference are not in real-units (for LAI -> [m2/m2]). Instead they are in transformed space, ranging from 0.0 (for dense vegetation) - 1.0 (for bare soil). This is highlighted below. 

Please also note that for LAI we use  relatively coarse resolution prior information. Therefore for retrievals over short time periods (particular locations with high cloud cover), this coarse resolution will be apparant.


LAI is not the only variable within the state-vector that are defined in transformed equivalents. The other transformed parameters are: leaf chlorophyll content (cab),leaf  water content (cw), leaf dry matter (cdm), and leaf average angle.

\begin{equation*}
    cab_{real}    = (-100.0)* ln( cab_{t} )
\end{equation*}

\begin{equation*}
    cw_{real}    = (-1/50.0)* log( data )
\end{equation*}

\begin{equation*}    
    cdm_{real}    = (-1/100.0)* log( cdm_{t} )
\end{equation*}

\begin{equation*}    
    LAI_{real}    = -2.0* log( LAI_{t} )
\end{equation*}

\begin{equation*}    
    ALA_{real}    = 90.0 * ALA_{t}
\end{equation*}

A Inverse Transformation module is provided within Tools.py to facilitate the retrieval of the actual values. 

In [ ]:
Data_t = InvTransformation(variable_of_interest,Data)
print('Transformed (min,mean,max) Values  are  [%3.2e, %3.2e, %3.2e]' % (np.nanmin(Data), np.nanmean(Data), np.nanmax(Data)))
print('Real (min,mean,max) Values are  [%3.2e, %3.2e, %3.2e]' % (np.nanmin(Data_t), np.nanmean(Data_t), np.nanmax(Data_t)))
      
Plot_Transformation(Data,Data_t)

## 6.4 Plot Temporal evolution

In [ ]:
Plot_TRAIT_evolution(Data_t,variable_of_interest)    

# 7. Download the Data
We first need to compile all the biophysical parameters into a single zip file.

In [ ]:
import os
os.system(f'tar -cvf {working_dir}/biophys_download.zip {biophys_output}')

In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks('download')


In [ ]:
toc = datetime.datetime.now()
T = (toc - tic).seconds
days = np.floor(T/60./60./24)
res = T - days*60.*60.*24.
hours = np.floor(res/60./60.)
res = T - hours
minutes = np.ceil(res/60.)

print('The run took %02d:%02d:%02d minutes ' % (days,hours,minutes))


# the Contributing team
<img src="pics/Logos.png">
